In [1]:
! pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.2 MB/s 
     |████████████████████████████████| 4.7 MB 51.8 MB/s 
     |████████████████████████████████| 141 kB 63.7 MB/s 
     |████████████████████████████████| 115 kB 67.4 MB/s 
     |████████████████████████████████| 212 kB 61.6 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
     |████████████████████████████████| 127 kB 49.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.1 MB/s 
     |████████████████████████████████| 1.3 MB 46.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!nvidia-smi

Mon Aug 15 23:52:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!wget -c https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11858/00-097C-0000-0023-65A9-5/urdu-plain-text-corpus.gz?sequence=2&isAllowed=y -O urdu-plain-text-corpus.gz

/bin/bash: -O: command not found
--2022-08-15 23:52:49--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11858/00-097C-0000-0023-65A9-5/urdu-plain-text-corpus.gz?sequence=2
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223826069 (213M) [application/x-gzip]
Saving to: ‘urdu-plain-text-corpus.gz?sequence=2’

urdu-plain-text-cor 100%[===================>] 213.46M  13.3MB/s    in 17s     

2022-08-15 23:53:08 (12.5 MB/s) - ‘urdu-plain-text-corpus.gz?sequence=2’ saved [223826069/223826069]



In [5]:
!gzip -d  /content/urdu-plain-text-corpus.gz

In [11]:
import os
import re
save_file_name="w2v_ur_"
path="./data/"
fpath = os.path.join(path, "/content/urdu-plain-text-corpus.txt")
f = open(fpath, encoding='utf-8')
t = f.read()

# text cleaning and pre-processing:
def delete_urdu_english_symbols(sentences):
    cleaned = []
    for sentence in sentences:
        text=re.sub(r"\n|\r", " ",sentence)
        text = re.sub(r"\d+", " ", text)
        # English punctuations
        text = re.sub(r"""[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]+""", " ", text)
        # Urdu punctuations
        text = re.sub(r"[:؛؟’‘٭ء،]+", " ", text)
        # Arabic numbers
        text = re.sub(r"[٠‎١‎٢‎٣‎٤‎٥‎٦‎٧‎٨‎٩]+", " ", text)
        text = re.sub(r"[^\w\s]", " ", text)
        # Remove English characters and numbers.
        text = re.sub(r"[a-zA-z0-9]+", " ", text)
        # remove multiple spaces.
        text = re.sub(r" +", " ", text)
        text = text.split(" ")
        # some stupid empty tokens should be removed.
        text = [t.strip() for t in text if t.strip()]
        cleaned.append(" ".join(text))
    return cleaned
text=re.sub(r"؟","۔",t)

raw_cleaned_text = delete_urdu_english_symbols(t.split("۔"))
# t[0:200]
print(raw_cleaned_text[:10])





['میرے بھائی کا ای میل آیاہے', 'بلال بھائی ہم آپ کی محنت کے معترف اور قائل ہیں', 'اکیس ستمبر دو ہزار آٹھ کو میریئٹ ہوٹل پر ٹرک خود کش حملے میں چون افراد ہلاک ہوئے جبکہ اڑھائی سو سے زائد زخمی ہوئے', 'محفوظات برائے خون کے آنسو ٹیگ شمشاد بھائ مجھے تو پتہ ہی نہیں تھا کہ محفل میں شادی دفتر بھی ہے اگر آپ کو زیادہ امیدوار کی ضرورت ہے تو میں بھی حاضر ہوں آخر ایک اور ایک گیارہ ہوتے ہیں يہ پچھلے پہر کا زرد رو چاند لفظ ندا عموما اول آتا ہے لیکن زور دینے کے لیے آخر مین بھی آجاتا ہے', 'مخدوم محمد حسین مادہ تاریخ وفات ہے', 'قصر امارت کے پاس پہنچے تو وہاں کے منظر سے انھیں کچھ خطرہ محسوس ہوا اس لئے وہ اسما بن خارجی کی طرف مڑے اور کہا بھائی مجھے اس شخص سے خوف آتا ہے اور میرا دل کہتا ہے کہ کچھ ہونے والا ہے', 'آخر اسٹوڈنٹ کس کی ہوں ان لوگوں نے مذہب کے نام پر اپنی اپنی دوکان داریاں بنا رکھی ہیں', 'اور ا سی نے ہر چیز کو پیدا فرمایا ہے پھر ا س کی بقا و ا رتقا کے ہر مرحلہ پر ا س کے خواص ا فعال اور مد ت ا لغرض ہر چیز کو ایک مقر رہ اندازے پر ٹھہرایا ہے عشا کی نماز میں قرا ت کے سلسلے میں معاذ بن جبل رضی اللہ عن

In [12]:
print(len(raw_cleaned_text))
raw_cleaned_text_test=raw_cleaned_text[1000000:1050000]
raw_cleaned_text_train=raw_cleaned_text[:1000000]


3169071


In [13]:
print("Test",len(raw_cleaned_text_test), "Train",len(raw_cleaned_text_train))

Test 50000 Train 1000000


In [17]:
batch_size = 512
all_texts = [raw_cleaned_text[i : i + batch_size] for i in range(0, len(raw_cleaned_text), batch_size)]


In [18]:
def batch_iterator():
    for i in range(0, len(raw_cleaned_text), batch_size):
        yield raw_cleaned_text[i : i + batch_size]

In [19]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
from tokenizers.normalizers import NFKC, Sequence

In [20]:
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = Sequence([
            NFKC()
        ])
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.decoder = decoders.ByteLevel()

In [21]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

In [22]:
trainer = trainers.BpeTrainer(vocab_size=9900000,min_frequency =2,show_progress =True, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
    "<|endoftext|>"
])
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

In [23]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [24]:
enc=new_tokenizer.encode(raw_cleaned_text[0],)
enc,raw_cleaned_text[0]

([1635, 384, 168, 424, 1505, 29517], 'میرے بھائی کا ای میل آیاہے')

In [25]:
enc

[1635, 384, 168, 424, 1505, 29517]

In [26]:
new_tokenizer.decode([1635, 384, 168, 424, 1505, 29517])

'میرے بھائی کا ای میل آیاہے'

In [27]:
raw_cleaned_text[0]

'میرے بھائی کا ای میل آیاہے'

In [28]:
new_tokenizer.save_pretrained("my-new-tokenizer")

('my-new-tokenizer/tokenizer_config.json',
 'my-new-tokenizer/special_tokens_map.json',
 'my-new-tokenizer/vocab.json',
 'my-new-tokenizer/merges.txt',
 'my-new-tokenizer/added_tokens.json',
 'my-new-tokenizer/tokenizer.json')

In [29]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [30]:
!git config --global credential.helper store

In [32]:
new_tokenizer.push_to_hub("gpt2-urdu-tokenizer")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='gpt2-urdu-tokenizer' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:644: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/hadidev/gpt2-urdu-tokenizer into local empty directory.


Download file tokenizer.json:   0%|          | 1.58k/21.2M [00:00<?, ?B/s]

Download file vocab.json:   0%|          | 3.48k/10.4M [00:00<?, ?B/s]

Clean file vocab.json:   0%|          | 1.00k/10.4M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/21.2M [00:00<?, ?B/s]

Upload file vocab.json:   0%|          | 3.34k/9.28M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/hadidev/gpt2-urdu-tokenizer
   b727688..ff35605  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/hadidev/gpt2-urdu-tokenizer
   b727688..ff35605  main -> main



'https://huggingface.co/hadidev/gpt2-urdu-tokenizer/commit/ff35605de34135f209ba9a6daeada7cae5dd9ac9'

In [33]:
new_tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=290869, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [34]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("hadidev/gpt2-urdu-tokenizer")

In [5]:
tokenizer.encode("م____یرےومر")

[122, 3640, 16687]

In [6]:
tokenizer.decode(tokenizer.encode("م____یرےومر"))

'میرےومر'

In [35]:
tokenizer.decode([1635, 384, 168, 424, 1505, 29517])

'میرے بھائی کا ای میل آیاہے'

In [36]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained("hadidev/gpt2-urdu-tokenizer")
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)


In [34]:
# model.config.to_json_file("config.json")

In [37]:
single_string = ''
for text in raw_cleaned_text_train:
  
  single_string += text + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [38]:
examples = []
block_size = 100
BATCH_SIZE = 16
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [39]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [ ]:
num_epoch = 30
history = model.fit(dataset, epochs=num_epoch)  

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:439: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 223387392 elements. This may consume a large amount of memory.
  num_elements)


11764/18078 [==================>...........] - ETA: 1:58:13 - loss: 6.0354 - accuracy: 0.1453

In [21]:
text = "سڑک پر"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  
  temperature = 0.7,
  no_repeat_ngram_size=2,num_beams=5, num_return_sequences=3, do_sample=True
)

Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


In [20]:
tokenizer.decode(beam_output[2])

'سڑک پر حملہ کیا گیا ہے'

In [ ]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
import os
output_dir = './gpt2_ur_small/'
# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained(output_dir)

('./gpt2_ur_small/tokenizer_config.json',
 './gpt2_ur_small/special_tokens_map.json',
 './gpt2_ur_small/vocab.json',
 './gpt2_ur_small/merges.txt',
 './gpt2_ur_small/added_tokens.json')

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model = TFGPT2LMHeadModel.from_pretrained(output_dir)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt2_ur_small/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
text = "سڑک پر"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 2,
  # temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=1
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


In [ ]:
tokenizer.decode(beam_output[0]).replace("</s>","")

'سڑک پر موجود ایک شخص نے بتایا کہ اس کے پاس کوئی چارہ نہیں ہے اور وہ اس کی مدد کر رہا ہے اس لیے اس نے اس کا نام ظاہر نہیں کیا اور نہ ہی اس پر کوئی پابندی عائد کی گئی ہے'

In [ ]:
# model.push_to_hub("gpt2-urdu-smallest")
tokenizer.push_to_hub("gpt2-urdu-smallest")

ValueError: ignored

In [ ]:
tokenizer.push_to_hub("gpt2-urdu-tokenizer-withgpt2")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='gpt2-urdu-tokenizer-withgpt2' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:644: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/hadidev/gpt2-urdu-tokenizer-withgpt2 into local empty directory.


Download file vocab.json:   0%|          | 7.58k/10.4M [00:00<?, ?B/s]

Clean file vocab.json:   0%|          | 1.00k/10.4M [00:00<?, ?B/s]

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("hadidev/gpt2-urdu-smallest", use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained("hadidev/gpt2-urdu-smallest", use_auth_token=True,from_tf=True)

All TF 2.0 model weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.


In [32]:
text = "سڑک پر"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 2,
  # temperature = 0.7,
  # no_repeat_ngram_size=2,
  # num_return_sequences=1
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValueError: ignored

In [ ]:
tokenizer.decode(beam_output[0]).replace("</s>","")

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model='hadidev/gpt2-urdu-smallest')
generator("මම", max_length=50, num_return_sequences=5)
